In [1]:
from collections import defaultdict
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

DIAS_MES_PADRAO = 30

def vfuturo(vp, i, n):
    return vp*(1+i)**n

def vpresente(vf, i, n):
    return vf/(1+i)**n

def taxa_mensal_para_dia(i):
    return (1 + i) ** (1/DIAS_MES_PADRAO) - 1

# Relação prestação x valor solicitado

$$ PV = PMT * FPV(i,n) $$

com:
 
$$ FPV(i, n) = \frac{1 - (1 + i)^{-n}}{i}$$

Onde:

- PV: Valor solicitado
- PMT: Valor da prestação
- n: Número de prestações
- i: taxa de jutos


## Informando o valor solicitado 

In [9]:
taxa_mensal = 0.015
valor_solicitado = 100_000
numero_de_prestacoes = 12
data_liberacao = date(2021, 1, 5)
dia_vencimento_parcela = 5
meses_de_carencia = 0

In [10]:
taxa_dia = taxa_mensal_para_dia(taxa_mensal)
taxa_dia

0.0004964102539346538

In [11]:
data_vecimento_carencia = [
    (data_liberacao + relativedelta(months=prestacao)).replace(day=dia_vencimento_parcela)
    for prestacao in range(1, meses_de_carencia + 1)
]

data_vencimento_parcelas = [
    (data_liberacao + relativedelta(months=prestacao)).replace(day=dia_vencimento_parcela)
    for prestacao in range(meses_de_carencia + 1, numero_de_prestacoes+meses_de_carencia + 1)
]


datas_vencimento = data_vecimento_carencia + data_vencimento_parcelas
pd.DataFrame({"mes": range(1, numero_de_prestacoes + meses_de_carencia + 1), "datas": datas_vencimento})

,mes,datas
0,1,2021-02-05
1,2,2021-03-05
2,3,2021-04-05
3,4,2021-05-05
4,5,2021-06-05
5,6,2021-07-05
6,7,2021-08-05
7,8,2021-09-05
8,9,2021-10-05
9,10,2021-11-05


In [12]:
total_dias = (datas_vencimento[-1] - data_liberacao).days
total_dias
print(f"{total_dias=}")

total_dias=365


In [13]:
saldo_devedor = valor_solicitado

tabela = defaultdict(list)

tabela['mes'].append(0)
tabela['saldo_devedor'].append(saldo_devedor)
tabela['amortizacao'].append(0)
tabela['juros'].append(0)
tabela['valor_prestacao'].append(0)
tabela['valor_presente_prestacao'].append(0)
tabela['dias_do_mes'].append(0)
tabela['dias_acumulados'].append(0)
tabela['data_vencimento'].append(0)
tabela['tipo'].append('Inicial')

for mes in range(1, meses_de_carencia + 1):

    dia_vencimento_atual = datas_vencimento[mes - 1]
    if mes == 1:
        dias_do_mes = (dia_vencimento_atual - data_liberacao).days
    else:
        dia_vencimento_anterior = datas_vencimento[mes - 2]
        dias_do_mes = (dia_vencimento_atual - dia_vencimento_anterior).days
    dias_acumulados = (dia_vencimento_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_do_mes) - saldo_devedor

    saldo_devedor *= (1+taxa_mensal)
    tabela['mes'].append(mes + 1)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(0)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(0)
    tabela['valor_presente_prestacao'].append(0)
    tabela['dias_do_mes'].append(dias_do_mes)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['data_vencimento'].append(dia_vencimento_atual)
    tabela['tipo'].append('carencia')

amortizacao = saldo_devedor / numero_de_prestacoes
print(f"{amortizacao=:10.2f}")

for mes in range(meses_de_carencia + 1, meses_de_carencia + numero_de_prestacoes+1):

    dia_vencimento_atual = datas_vencimento[mes - 1]
    if mes == 1:
        dias_do_mes = (dia_vencimento_atual - data_liberacao).days
    else:
        dia_vencimento_anterior = datas_vencimento[mes - 2]
        dias_do_mes = (dia_vencimento_atual - dia_vencimento_anterior).days
    dias_acumulados = (dia_vencimento_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_do_mes) - saldo_devedor
    valor_da_prestacao = amortizacao + juros
    saldo_devedor -= amortizacao

    tabela['mes'].append(mes)
    tabela['data_vencimento'].append(dia_vencimento_atual)
    tabela['dias_do_mes'].append(dias_do_mes)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(amortizacao)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(valor_da_prestacao)
    tabela['valor_presente_prestacao'].append(vpresente(valor_da_prestacao, taxa_dia, dias_acumulados))
    tabela['tipo'].append('parcela')


amortizacao=   8333.33


In [7]:
tabela_valor_solicidado_dado = pd.DataFrame(tabela)
tabela_valor_solicidado_dado.style.format({
    'dias_parcela': '{:d}',
    'dias_acumulados': '{:d}',
    'saldo_devedor': '{:.2f}',
    'amortizacao': '{:.2f}',
    'juros': '{:.2f}',
    'valor_prestacao': '{:.2f}',
    'valor_presente_prestacao': '{:.2f}'
})

,mes,saldo_devedor,amortizacao,juros,valor_prestacao,valor_presente_prestacao,dias_do_mes,dias_acumulados,data_vencimento,tipo
0,0,100000.00,0.00,0.00,0.00,0.00,0,0,0,Inicial
1,1,90000.00,10000.00,1550.39,11550.39,11374.04,31,31,2021-02-05,parcela
2,2,80000.00,10000.00,1259.37,11259.37,10934.47,28,59,2021-03-05,parcela
3,3,70000.00,10000.00,1240.31,11240.31,10749.30,31,90,2021-04-05,parcela
4,4,60000.00,10000.00,1050.00,11050.00,10411.14,30,120,2021-05-05,parcela
5,5,50000.00,10000.00,930.23,10930.23,10141.07,31,151,2021-06-05,parcela
6,6,40000.00,10000.00,750.00,10750.00,9826.45,30,181,2021-07-05,parcela
7,7,30000.00,10000.00,620.15,10620.15,9559.55,31,212,2021-08-05,parcela
8,8,20000.00,10000.00,465.12,10465.12,9276.18,31,243,2021-09-05,parcela
9,9,10000.00,10000.00,300.00,10300.00,8994.90,30,273,2021-10-05,parcela


In [8]:
tabela_valor_solicidado_dado[['amortizacao', 'juros', 'valor_prestacao', 'valor_presente_prestacao']].sum()

amortizacao                 100000.000000
juros                         8320.607385
valor_prestacao             108320.607385
valor_presente_prestacao    100000.000000
dtype: float64